Use adata output of the 02 notebook to make plots

In [1]:
import os
import glob
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scanpy as sc
from pathlib import Path
import h5py
import re
import pickle

from hest.vis.plot import *
from hest.vis.visualise_pred import *

In [2]:
RUN_ROOT = "/project/simmons_hts/kxu/hest/eval/ST_pred_results"
df_summary = summarize_runs(RUN_ROOT)
df_summary.to_csv("/project/simmons_hts/kxu/hest/eval/experiment_log.csv")
#display(df_summary)
best_df = best_results_by_gene_and_dataset(df_summary)
# Remove rows where dataset == "pilot" (case insensitive, just in case)
# best_df = best_df[best_df["dataset"].str.lower() != "pilot"].reset_index(drop=True)
best_df

/home/k/kxu/.local/lib/python3.11/site-packages/hest/vis/plot.py:543: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  .idxmax()
/home/k/kxu/.local/lib/python3.11/site-packages/hest/vis/plot.py:543: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  .idxmax()
/home/k/kxu/.local/lib/python3.11/site-packages/hest/vis/plot.py:543: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  .idxmax()
/home/k/kxu/.local/lib/python3.11/site-packages/hest/vis/plot.py:543: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  .idxmax()


,run,gene_list,num_genes,alpha,batch_size,dimreduce,encoders,normalize,library_size_normalize,latent_dim,method,dataset,best_model,highest_pearson_mean,highest_pearson_std,num_training_patches_mean
0,run_25-09-09-11-50-04,all_genes.json,5100.0,None,128.0,PCA,"uni_v1, resnet50, ctranspath, phikon, phikon_v...",True,False,256.0,ridge,XeniumPR1,virchow2,0.510421,0.209865,25137.466667
1,run_25-10-03-14-11-52,all_genes.json,312.0,None,256.0,PCA,"uni_v1, resnet50, ctranspath, phikon, phikon_v...",True,False,256.0,ridge,XeniumPR1_broad,virchow2,0.512409,0.050549,33023.500000
2,run_25-10-08-10-24-13,all_genes.json,5100.0,None,256.0,PCA,"uni_v2, uni_v1, resnet50, ctranspath, phikon, ...",True,False,256.0,ridge,XeniumPR1_segger,hoptimus1,0.512151,0.210870,24841.600000
3,run_25-09-20-19-30-20,all_genes.json,460.0,None,128.0,PCA,"uni_v1, resnet50, ctranspath, phikon, phikon_v...",True,False,256.0,ridge,broad,hoptimus0,0.674139,0.031086,33798.000000
4,run_25-09-05-16-12-17,cell_specific_genes.json,47.0,None,128.0,PCA,"uni_v1, resnet50, ctranspath, phikon, phikon_v...",True,False,256.0,ridge,XeniumPR1,hoptimus0,0.602295,0.097945,25137.466667
5,run_25-10-03-16-02-25,filtered_genes_minpct10.json,3065.0,None,256.0,PCA,"uni_v1, resnet50, ctranspath, phikon, phikon_v...",True,False,256.0,ridge,XeniumPR1_segger,virchow2,0.640648,0.120953,24841.600000
6,run_25-09-06-14-12-40,full_panel_genes.json,363.0,None,128.0,PCA,"uni_v1, resnet50, ctranspath, phikon, phikon_v...",True,False,256.0,ridge,XeniumPR1,virchow2,0.526018,0.086170,25137.466667
7,run_25-09-03-17-04-11,var_50genes.json,50.0,None,128.0,PCA,"resnet50, ctranspath, phikon, phikon_v2, kaiko...",True,False,256.0,ridge,XeniumPR1,hoptimus0,0.497417,0.081020,25137.466667
8,run_25-09-03-18-35-49,var_50genes.json,50.0,None,128.0,PCA,"uni_v1, resnet50, ctranspath, phikon, phikon_v...",True,False,256.0,ridge,pilot,gigapath,0.300399,0.004557,857.000000
9,run_25-09-24-17-32-01,var_genes.json,NaN,None,128.0,PCA,"uni_v1, resnet50, h0_mini, hibou_large, kaiko_...",True,False,256.0,ridge,broad_cell_centered,hibou_large,0.197328,0.143339,910801.714286


# Load adata_list with predicted values added

In [3]:
RUN = 'run_25-10-08-10-24-13'
print("Selected RUNS:", RUN)

best_model, dataset, df_corr = extract_best_model_gene_corrs(RUN)

Selected RUNS: run_25-10-08-10-24-13
[extract] dataset=XeniumPR1_segger encoder=hoptimus1 pearson_mean=0.5121510545390398 std=0.21087012787193857


In [5]:
data_dir = os.path.join("/project/simmons_hts/kxu/hest/eval/ST_pred_results", RUN, "pred")
adata_list = []
sample_names = []

for fname in os.listdir(data_dir):
    if not fname.endswith(".h5ad"):
        continue
    sample = os.path.splitext(fname)[0]
    fpath = os.path.join(data_dir, fname)
    adata = sc.read_h5ad(fpath)
    # store sample name in .obs (same value for all rows)
    adata.obs['sample_id'] = sample
    adata_list.append(adata)
    sample_names.append(sample)

print("Loaded and labeled samples:", sample_names)
print(f"Loaded {len(adata_list)} samples")
print(f"First sample shape: {adata_list[0].shape}")

Loaded and labeled samples: ['XeniumPR1S1ROI8', 'XeniumPR1S1ROI3', 'XeniumPR1S2ROI3', 'XeniumPR1S2ROI5', 'XeniumPR1S2ROI2', 'XeniumPR1S1ROI5', 'XeniumPR1S1ROI7', 'XeniumPR1S1ROI4', 'XeniumPR1S1ROI2', 'XeniumPR1S2ROI7', 'XeniumPR1S1ROI1', 'XeniumPR1S2ROI1', 'XeniumPR1S2ROI6', 'XeniumPR1S1ROI6', 'XeniumPR1S2ROI4']
Loaded 15 samples
First sample shape: (4687, 5161)


In [6]:
adata_list

[AnnData object with n_obs × n_vars = 4687 × 5161
     obs: 'in_tissue', 'pxl_col_in_fullres', 'pxl_row_in_fullres', 'array_col', 'array_row', 'n_counts', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'sample_id'
     var: 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
     uns: 'spatial'
     obsm: 'spatial'
     layers: 'pred', 'target',
 AnnData object with n_obs × n_vars = 1611 × 5161
     obs: 'in_tissue', 'pxl_col_in_fullres', 'pxl_row_in_fullres', 'array_col', 'array_row', 'n_counts', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'sample_id'
     var: 'n_cells_by_counts', 'mean_counts', '

# plot pred

In [7]:
cell_specific_genes = load_gene_list('XeniumPR1', 'cell_specific_genes.json')
cell_specific_genes[1:4]

conditional_genes = load_gene_list('XeniumPR1', 'conditional_genes.json')
conditional_genes[1:4]

Loaded cell_specific_genes.json for dataset 'XeniumPR1'
Loaded conditional_genes.json for dataset 'XeniumPR1'


['REG1A', 'IL6', 'MUC5B']

In [17]:
results = save_spatial_pred_target_pdfs_for_adata_list(
    adata_list,
    cell_specific_genes,
    RUN_ROOT=RUN_ROOT,
    RUN=RUN,
    folder_name="cell_specific_genes",
    img_key="downscaled_fullres",
    pred_layer="pred",
    target_layer="target",
    genes_per_page=4,
    size=1.2,
    dpi=100,
    cmap=None,
    verbose=True
)

[START] Writing PDF for adata_list[0] (XeniumPR1S1ROI8) -> /project/simmons_hts/kxu/hest/eval/ST_pred_results/run_25-10-08-10-24-13/plots/pred/cell_specific_genes/XeniumPR1S1ROI8.pdf (genes: 47)
[OK] Wrote PDF: /project/simmons_hts/kxu/hest/eval/ST_pred_results/run_25-10-08-10-24-13/plots/pred/cell_specific_genes/XeniumPR1S1ROI8.pdf
[START] Writing PDF for adata_list[1] (XeniumPR1S1ROI3) -> /project/simmons_hts/kxu/hest/eval/ST_pred_results/run_25-10-08-10-24-13/plots/pred/cell_specific_genes/XeniumPR1S1ROI3.pdf (genes: 47)
[OK] Wrote PDF: /project/simmons_hts/kxu/hest/eval/ST_pred_results/run_25-10-08-10-24-13/plots/pred/cell_specific_genes/XeniumPR1S1ROI3.pdf
[START] Writing PDF for adata_list[2] (XeniumPR1S2ROI3) -> /project/simmons_hts/kxu/hest/eval/ST_pred_results/run_25-10-08-10-24-13/plots/pred/cell_specific_genes/XeniumPR1S2ROI3.pdf (genes: 47)
[OK] Wrote PDF: /project/simmons_hts/kxu/hest/eval/ST_pred_results/run_25-10-08-10-24-13/plots/pred/cell_specific_genes/XeniumPR1S2ROI3

In [18]:
conditional_genes = load_gene_list('XeniumPR1', 'conditional_genes.json')
conditional_genes[1:4]

Loaded conditional_genes.json for dataset 'XeniumPR1'


['REG1A', 'IL6', 'MUC5B']

In [19]:
results = save_spatial_pred_target_pdfs_for_adata_list(
    adata_list,
    conditional_genes,
    RUN_ROOT=RUN_ROOT,
    RUN=RUN,
    folder_name="conditional_genes",
    img_key="downscaled_fullres",
    pred_layer="pred",
    target_layer="target",
    genes_per_page=4,
    size=1.2,
    dpi=100,
    cmap=None,
    verbose=True
)

[START] Writing PDF for adata_list[0] (XeniumPR1S1ROI8) -> /project/simmons_hts/kxu/hest/eval/ST_pred_results/run_25-10-08-10-24-13/plots/pred/conditional_genes/XeniumPR1S1ROI8.pdf (genes: 24)
[OK] Wrote PDF: /project/simmons_hts/kxu/hest/eval/ST_pred_results/run_25-10-08-10-24-13/plots/pred/conditional_genes/XeniumPR1S1ROI8.pdf
[START] Writing PDF for adata_list[1] (XeniumPR1S1ROI3) -> /project/simmons_hts/kxu/hest/eval/ST_pred_results/run_25-10-08-10-24-13/plots/pred/conditional_genes/XeniumPR1S1ROI3.pdf (genes: 24)
[OK] Wrote PDF: /project/simmons_hts/kxu/hest/eval/ST_pred_results/run_25-10-08-10-24-13/plots/pred/conditional_genes/XeniumPR1S1ROI3.pdf
[START] Writing PDF for adata_list[2] (XeniumPR1S2ROI3) -> /project/simmons_hts/kxu/hest/eval/ST_pred_results/run_25-10-08-10-24-13/plots/pred/conditional_genes/XeniumPR1S2ROI3.pdf (genes: 24)
[OK] Wrote PDF: /project/simmons_hts/kxu/hest/eval/ST_pred_results/run_25-10-08-10-24-13/plots/pred/conditional_genes/XeniumPR1S2ROI3.pdf
[START]